In [3]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00


In [6]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9666dcfef091d6dffbe6dcd306bc74011ded50ede25af689eee5965201b35fe0
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

base_model_name = "google/mt5-base"

ft_model_name = "suryakantmani/mt5-vanilla-finetune-summarization"

tokenizer = AutoTokenizer.from_pretrained(ft_model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name).to(device)
ft_model = AutoModelForSeq2SeqLM.from_pretrained(ft_model_name).to(device)

rouge = evaluate.load("rouge")

eval_data = [
    {
        "src": "भारत एक बड़ा देश है जिसमें कई राज्य हैं। यह एशिया में स्थित है और इसकी राजधानी नई दिल्ली है।",
        "ref": "भारत एशिया में स्थित एक बड़ा देश है।"
    },
    {
        "src": "La Terre tourne autour du Soleil et c’est ce mouvement qui crée les saisons sur notre planète.",
        "ref": "La rotation de la Terre autour du Soleil cause les saisons."
    },
    {
        "src": "The ozone layer protects Earth from harmful ultraviolet rays from the Sun.",
        "ref": "The ozone layer shields Earth from UV rays."
    }
]

def generate_summary(model, text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=80, num_beams=4)
    return tokenizer.decode(out[0], skip_special_tokens=True)

def evaluate_model(model, name):
    preds, refs = [], []
    for sample in eval_data:
        pred = generate_summary(model, sample["src"])
        print(f"\n[{name}] Input: {sample['src']}\n→ Pred: {pred}\nRef: {sample['ref']}")
        preds.append(pred)
        refs.append(sample["ref"])
    scores = rouge.compute(predictions=preds, references=refs)
    rouge_l = scores["rougeL"]  # updated for new API
    print(f"\n{name} ROUGE-L: {rouge_l:.4f}")
    return rouge_l


base_score = evaluate_model(base_model, "BASE mT5")
ft_score = evaluate_model(ft_model, "FINE-TUNED mT5")

print(f"\n Catastrophic Forgetting (ΔROUGE-L) = {base_score - ft_score:.4f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MT5Tokenizer'. 
The class this function is called from is 'T5Tokenizer'.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



[BASE mT5] Input: भारत एक बड़ा देश है जिसमें कई राज्य हैं। यह एशिया में स्थित है और इसकी राजधानी नई दिल्ली है।
→ Pred:  <extra_id_0> नई दिल्ली
Ref: भारत एशिया में स्थित एक बड़ा देश है।

[BASE mT5] Input: La Terre tourne autour du Soleil et c’est ce mouvement qui crée les saisons sur notre planète.
→ Pred:  <extra_id_0>. La Terre tourne...
Ref: La rotation de la Terre autour du Soleil cause les saisons.

[BASE mT5] Input: The ozone layer protects Earth from harmful ultraviolet rays from the Sun.
→ Pred:  <extra_id_0> ozone layer
Ref: The ozone layer shields Earth from UV rays.

BASE mT5 ROUGE-L: 0.1810

[FINE-TUNED mT5] Input: भारत एक बड़ा देश है जिसमें कई राज्य हैं। यह एशिया में स्थित है और इसकी राजधानी नई दिल्ली है।
→ Pred:  'भारत' - 'भारत - 'भारत-एक बड़ा देश' - 'भारत - 'भारत - 'भारत - 'भारत - '
Ref: भारत एशिया में स्थित एक बड़ा देश है।

[FINE-TUNED mT5] Input: La Terre tourne autour du Soleil et c’est ce mouvement qui crée les saisons sur notre planète.
→ Pred:  La Terre tourne auto